In [1]:
import fitbit

In [2]:
api = fitbit.API(debug=True)

Visit the following url to get your auth code: https://www.fitbit.com/oauth2/authorize?response_type=code&client_id=23BB84&redirect_uri=http%3A%2F%2Flocalhost&scope=activity+nutrition+heartrate+location+nutrition+profile+settings+sleep+social+weight
Auth code: 11cf4117fa9419a3e62dd33a33b3c85fad29521d


In [8]:
res = api.devices()
res.json()

[{'battery': 'Low',
  'batteryLevel': 13,
  'deviceVersion': 'Charge 3',
  'features': [],
  'id': '986173408',
  'lastSyncTime': '2021-09-10T13:57:34.000',
  'mac': '83DA14176FC3',
  'type': 'TRACKER'}]

In [35]:
res = api.add_alarm(tracker, "11:30-05:00", False, False, "MONDAY")
res.json()

{'trackerAlarm': {'alarmId': 1892535475,
  'deleted': False,
  'enabled': False,
  'recurring': False,
  'snoozeCount': 3,
  'snoozeLength': 9,
  'syncedToDevice': True,
  'time': '11:30-05:00',
  'vibe': 'DEFAULT',
  'weekDays': []}}

In [33]:
res = api.delete_alarm(tracker, 1892511157)
res

<Response [204]>

In [34]:
res.text

''